In [16]:
# import tensorflow.compat.v1 as tf
# from tensorflow.compat.v1 import keras
from os import listdir
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imsave, imshow
import numpy as np
import pandas as pd
import math
from scipy.signal import convolve2d
from skimage import img_as_ubyte, img_as_float
from skimage.transform import resize
# from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# from keras import Sequential
# from keras import layers
import copy
from skimage.color import rgb2gray
from sklearn.metrics import mean_squared_error as MSE
import gc
import os.path
gc.enable()

# tf.keras.backend.clear_session()  # For easy reset of notebook state.
# sess = tf.Session() 
# keras.backend.set_session(sess)


def make_ker(ker_len, ker_angle):
#     h = ker_len * 2
    h = ker_len
    ker_len = ker_len // 2
    ker = np.zeros((h, h), dtype='float')
    k = -math.tan(ker_angle)
    b = (1 - k) * ker_len
    if abs(abs(ker_angle * 180/math.pi) - 90) > 10:
        for x in range(h):
            y = round(k * x + b)
            y = int((y if y >= 0 else 0) if y <= h-1 else h-1)
            if (y == 0 or y == h - 1):
                continue
            ker[y, x] = 1
    else:
        for y in range(h):
            ker[y, ker_len] = 1 
    ret_value = ker/ker.sum()
    if np.isnan(np.sum(ret_value)):
        return []
    else:
        return ret_value

In [17]:
import torch

In [18]:
# from tensorflow.python.client import device_lib

# def get_available_gpus():
#     local_device_protos = device_lib.list_local_devices()
#     return [x.name for x in local_device_protos if x.device_type == 'GPU']
# get_available_gpus()

In [19]:
import random
from skimage.transform import SimilarityTransform
from skimage.transform import warp
from skimage.util import random_noise

IMG_SIZE = 200
BLUR_LEN = 5
# IMG_SIZE = None
# OUT_SIZE = 600
BATCH_SIZE = 8
EPS = 10e-7

def rotate_transform_matrix(transform):
    """Rotate matrix so it can be applied to row:col coordinates."""
    matrix = transform.params[(1, 0, 2), :][:, (1, 0, 2)]
    return type(transform)(matrix)

def prepare_img(img):
    image = copy.copy(img)
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
    if (image.shape[2] == 4):
        image = image[:,:,:3]
    h, w = image.shape[:2]
    return image

def prepare_img1(img):
    image = copy.copy(img)
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
    if (image.shape[2] == 4):
        image = image[:,:,:3]
    return img_as_float(image)

# train_generator = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1)
# train_generator.fit(X, augment=True, seed=42)


In [20]:
import random

In [21]:
IMG_SIZE = 600
BLUR_LEN = 10
KER = make_ker(BLUR_LEN, 0)
global_idx = 0

def generate_pics(paths, amount_on_picture, break_on = 0):
    global global_idx
    for p in paths:
        try:
            cur_img = prepare_img1(imread(p))
            blurred = convolve_img(cur_img)
            noised = random_noise(blurred, var=0.001)
            h, w = cur_img.shape[:2]
            if (h > IMG_SIZE and w > IMG_SIZE):
                for idx in range(amount_on_picture):
                    y = random.randrange(h - IMG_SIZE)
                    x = random.randrange(w - IMG_SIZE)
                    plt.imsave('pict/random_la/bn/img_' + str(global_idx) + '.png', noised[y : y + IMG_SIZE, x : x + IMG_SIZE])
                    plt.imsave('pict/random_la/sn/img_' + str(global_idx) + '.png', cur_img[y : y + IMG_SIZE, x : x + IMG_SIZE])
                    global_idx += 1
            if (break_on != 0):
                if (break_on == global_idx):
                    return
        except:
            pass

def generate_pics_from_two(paths_X, paths_y, amount_on_picture):
    global_idx = 0
    for p in range(len(paths_X)):
        blurred = prepare_img1(imread(paths_X[p]))
        cur_img = prepare_img1(imread(paths_y[p]))
        h, w = cur_img.shape[:2]
        for idx in range(amount_on_picture):
            y = random.randrange(h - IMG_SIZE)
            x = random.randrange(w - IMG_SIZE)
            plt.imsave('pict/b_test/test_' + str(global_idx) + '.png', blurred[y : y + IMG_SIZE, x : x + IMG_SIZE])
            plt.imsave('pict/s_test/test_' + str(global_idx) + '.png', cur_img[y : y + IMG_SIZE, x : x + IMG_SIZE])
            global_idx += 1
    return

def convolve_img(image):
    # ker = KER
        ker = make_ker(random.randint(2, 15), random.uniform(0, np.pi))
        ker_len = BLUR_LEN
        pad = ker_len // 2
        img = image
        if len(img.shape) == 2:
            img = np.stack((image, image, image))
            img = np.transpose(image, axes=(1,2, 0))
        # for channel in range(3):
            # image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
        r = convolve2d(np.pad(img[:,:,0], pad, 'edge'), ker, mode='valid'); #r /= r.max()
        g = convolve2d(np.pad(img[:,:,1], pad, 'edge'), ker, mode='valid'); #g /= g.max()
        b = convolve2d(np.pad(img[:,:,2], pad, 'edge'), ker, mode='valid'); #b /= b.max()
        image = np.stack((r, g, b))
        image = np.clip(image, 0., 1.)
        image = np.transpose(image, axes=(1,2, 0))
        return image
        print(ker)

In [22]:
# generate_pics_from_two(X, y, 1)
# generate_pics(y, 1, break_on=300)

In [23]:
%%time
train_dir = '../course/motion_blurred/'
# train_dir = 'b/'
y_dir = 'pict/google_initial/'
# y_dir = 'pict/s_/'
# y_dir = 's/'
fnames_X = listdir(train_dir)
fnames_y = listdir(y_dir)

X = list([train_dir + item for item in fnames_X])
y = list([y_dir + item for item in fnames_y])

Wall time: 80 ms


In [24]:
from sklearn.metrics import mean_squared_error as MSE
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR

def calc_metrics(predicted_dir, target_dir, list_len = None):
    pred_files = listdir(predicted_dir)
    targ_files = listdir(target_dir)
    if (not list_len is None):
        pred_files = pred_files[:list_len]
    l = len(pred_files)
    metrics = dict()
    metrics['PSNR'] = 0
    metrics['SSIM'] = 0
    for idx in range(len(pred_files)):
        print(pred_files[idx])
        p = prepare_img1(imread(os.path.join(predicted_dir, pred_files[idx])))
        if (pred_files[idx] in targ_files):
            t = prepare_img1(imread(os.path.join(target_dir, pred_files[idx])))
        try:
            metrics['PSNR'] += PSNR(t,p)
            metrics['SSIM'] += SSIM(t,p, multichannel=True)
        except:
            l -= 1

    metrics['PSNR'] /= l
    metrics['SSIM'] /= l
    return metrics


In [28]:
m0 = calc_metrics('pict/bn_val', 'pict/sn_val')

In [9]:
m1 = calc_metrics('pict/sn_val', 'result/bn_val_orig')

In [10]:
m2 = calc_metrics('pict/sn_val', 'result/bn_val_6')

In [11]:
m3 = calc_metrics('pict/sn_val', 'result/bn_val_20')

In [31]:
m4 = calc_metrics('pict/sn_val', 'pict/predicted_val')

In [33]:
m5 = calc_metrics('pict/sn_val', 'pict/predicted_val1')

In [30]:
print('Метрики на датасете:')
print('Сравнение размытых изображений с изначальными:')
print('  ' + str(m0))
print('Восстановленные Inception без дообучения:')
print('  ' + str(m1))
print('Восстановленное Inception после дообучения (6 эпох):')
print('  ' + str(m2))
print('Восстановленное Inception после дообучения (20 эпох):')
print('  ' + str(m3))

Метрики на датасете:
Сравнение размытых изображений с изначальными:
  {'PSNR': 23.425286662747613, 'SSIM': 0.6817486902035588}
Восстановленные Inception без дообучения:


NameError: name 'm1' is not defined

In [17]:
dirs = os.listdir('pict/random_la')
dirs.remove('bn')#, 'predicted', 'sn'])
dirs.remove('predicted')
dirs.remove('sn')

In [18]:
dirs

['deconv[0.001, 0.1, 0.001, 0.1]',
 'deconv[0.001, 0.1, 0.1, 0.001]',
 'deconv[0.1, 0.0001, 1, 1]',
 'deconv[0.1, 0.001, 0.001, 0.1]',
 'deconv[0.1, 0.001, 0.1, 0.001]',
 'deconv_001_01_1_1']

In [10]:
base_metrics = calc_metrics('pict/random_la/bn', 'pict/random_la/sn', 30)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [11]:
print(base_metrics)

{'PSNR': 21.861982620367282, 'SSIM': 0.6236894717602322}


In [28]:
%%time
m = dict()
for d in dirs:
    m[d] = calc_metrics(os.path.join('pict/random_la', d), 'pict/random_la/sn')

ValueError: Input images must have the same dimensions.

In [35]:
for key in m:
    print(key)
    print(m[key])

deconv[0.001, 0.1, 0.001, 0.1]
{'PSNR': 22.964748306501548, 'SSIM': 0.7424172859549522}
deconv[0.001, 0.1, 0.1, 0.001]
{'PSNR': 22.964748306501548, 'SSIM': 0.7424172859549522}
deconv[0.1, 0.0001, 1, 1]
{'PSNR': 22.94859650705285, 'SSIM': 0.7360348978859029}
deconv[0.1, 0.001, 0.001, 0.1]
{'PSNR': 22.964748306501548, 'SSIM': 0.7424172859549522}
deconv[0.1, 0.001, 0.1, 0.001]
{'PSNR': 22.964748306501548, 'SSIM': 0.7424172859549522}


In [27]:
neural_metrics = calc_metrics('pict/random_la/predicted', 'pict/random_la/sn', 31)
print(neural_metrics)

img_0.png
img_1.png
img_10.png
img_100.png
img_101.png
img_102.png
img_103.png
img_104.png
img_105.png
img_106.png
img_107.png
img_108.png
img_109.png
img_11.png
img_110.png
img_111.png
img_112.png
img_113.png
img_114.png
img_115.png
img_116.png
img_117.png
img_118.png
img_119.png
img_12.png
img_120.png
img_121.png
img_122.png
img_123.png
img_124.png
img_125.png
{'PSNR': 22.217719104460638, 'SSIM': 0.6563145783553344}


In [25]:
a = prepare_img1( plt.imread('pict/random_la/predicted/img_0.png'))
b = prepare_img1(plt.imread('pict/random_la/sn/img_0.png'))
PSNR(a,b)

16.801526179330935